In [1]:
import pandas as pd # type: ignore
import numpy as np # type: ignore
import matplotlib.pyplot as plt # type: ignore
import seaborn as sns # type: ignore

import statsmodels.api as sm # type: ignore
import statsmodels.formula.api as smf # type: ignore
from statsmodels.stats.outliers_influence import variance_inflation_factor as vif # type: ignore
from stargazer.stargazer import Stargazer # type: ignore

from IPython.display import display, HTML # type: ignore


In [2]:
# Bring in the data
df = pd.read_csv('../data/HuntersGreenHomeSales_prepped.csv')

# Have a peak
df.head()


,slnoskm,Status,Address,Street_only,lat,long,Beds,bathsfull,bathshalf,bathstotal,...,datesold,ds_dom,ds_moy,ds_year,splsale,splsal_Auction,splsal_shortsale,splsale_BankOwnedREO,pricesold,days_on_market
0,1,Sold,17711 ESPRIT DR,ESPRIT DR,28.127921,-82.340184,3,2,0,2,...,10/6/2018,6,10,2018,Short Sale,0,1,0,141500,451
1,2,Sold,17769 ESPRIT DR,ESPRIT DR,28.130015,-82.339863,2,2,0,2,...,7/30/2015,30,7,2015,NaN,0,0,0,160000,28
2,3,Sold,17622 ESPRIT DR,ESPRIT DR,28.129953,-82.341055,3,2,0,2,...,7/29/2015,29,7,2015,NaN,0,0,0,161500,42
3,4,Sold,18111 ASHTON PARK WAY,ASHTON PARK WAY,28.138067,-82.333513,3,2,0,2,...,7/29/2016,29,7,2016,NaN,0,0,0,170000,35
4,5,Sold,9203 CELEBRATION CT,CELEBRATION CT,28.129131,-82.340659,4,2,1,3,...,9/30/2016,30,9,2016,Short Sale,0,1,0,170000,417


In [3]:
# show me null data
df.isnull().sum()

slnoskm                     0
Status                      0
Address                     0
Street_only                 0
lat                         0
long                        0
Beds                        0
bathsfull                   0
bathshalf                   0
bathstotal                  0
sqft                        0
garages                     4
roof_Built-Up               0
roof_Concrete               0
roof_Shake                  0
roof_Shingle                0
roof_Slate                  0
roof_Tile                   0
roof_Other                  0
lotsqft                     0
yrblt                       0
pool_Community              0
pool_Private                0
pool_None                   0
spa                       313
subdivn                     0
adom_agentdaysonmarket      0
cdom_cumuldaysmls           0
listprice                   0
lppersqft                   0
PendingDate                 0
pd_dom                      0
pd_moy                      0
pd_year   

Let's drop the spa and splsale columns before we drop null rows so we don't lose a ton of data.  splsale has been recaptured by breakout columns.

In [4]:
df = df.drop(['spa', 'splsale'], axis=1)
df = df.dropna()

In [5]:
df.describe()


,slnoskm,lat,long,Beds,bathsfull,bathshalf,bathstotal,sqft,garages,roof_Built-Up,...,pd_year,sppersqft,ds_dom,ds_moy,ds_year,splsal_Auction,splsal_shortsale,splsale_BankOwnedREO,pricesold,days_on_market
count,478.000000,478.000000,478.000000,478.000000,478.000000,478.000000,478.000000,478.000000,478.000000,478.000000,...,478.000000,478.000000,478.000000,478.000000,478.000000,478.000000,478.000000,478.000000,4.780000e+02,478.000000
mean,241.834728,28.131651,-82.342580,3.864017,2.656904,0.341004,2.997908,2639.857741,2.493724,0.002092,...,2016.930962,129.312594,17.543933,6.675732,2016.995816,0.006276,0.016736,0.041841,3.466532e+05,42.104603
std,139.491364,0.004647,0.008928,0.703631,0.687919,0.496141,0.830496,827.257195,0.552261,0.045739,...,1.375681,21.024089,9.243342,3.106340,1.364411,0.079056,0.128416,0.200435,1.523696e+05,32.869055
min,1.000000,28.120558,-82.361649,2.000000,2.000000,0.000000,2.000000,1305.000000,1.000000,0.000000,...,2015.000000,73.850000,1.000000,1.000000,2015.000000,0.000000,0.000000,0.000000,1.415000e+05,0.000000
25%,121.250000,28.128696,-82.349495,3.000000,2.000000,0.000000,3.000000,2079.000000,2.000000,0.000000,...,2016.000000,114.962500,10.000000,4.000000,2016.000000,0.000000,0.000000,0.000000,2.479250e+05,30.000000
50%,242.500000,28.130965,-82.340279,4.000000,3.000000,0.000000,3.000000,2529.000000,2.000000,0.000000,...,2017.000000,129.595000,17.500000,7.000000,2017.000000,0.000000,0.000000,0.000000,3.277500e+05,38.000000
75%,361.750000,28.134790,-82.335950,4.000000,3.000000,1.000000,3.000000,3009.500000,3.000000,0.000000,...,2018.000000,142.227500,26.000000,9.000000,2018.000000,0.000000,0.000000,0.000000,3.965000e+05,48.000000
max,482.000000,28.141323,-82.327339,6.000000,6.000000,2.000000,8.000000,8398.000000,5.000000,1.000000,...,2019.000000,218.160000,31.000000,12.000000,2020.000000,1.000000,1.000000,1.000000,1.435000e+06,451.000000


In [6]:
# Look at normality
# plt.figure(figsize=(8, 5))
# sns.scatterplot(x=df['sqft'], y=df['pricesold'])
# plt.show()


In [7]:
# let's transform
df['log_sqft'] = np.log1p(df['sqft'])
df['house_age'] = df['pd_year'] - df['yrblt']
df.head()

,slnoskm,Status,Address,Street_only,lat,long,Beds,bathsfull,bathshalf,bathstotal,...,ds_dom,ds_moy,ds_year,splsal_Auction,splsal_shortsale,splsale_BankOwnedREO,pricesold,days_on_market,log_sqft,house_age
0,1,Sold,17711 ESPRIT DR,ESPRIT DR,28.127921,-82.340184,3,2,0,2,...,6,10,2018,0,1,0,141500,451,7.436028,25
1,2,Sold,17769 ESPRIT DR,ESPRIT DR,28.130015,-82.339863,2,2,0,2,...,30,7,2015,0,0,0,160000,28,7.174724,23
2,3,Sold,17622 ESPRIT DR,ESPRIT DR,28.129953,-82.341055,3,2,0,2,...,29,7,2015,0,0,0,161500,42,7.259116,24
3,4,Sold,18111 ASHTON PARK WAY,ASHTON PARK WAY,28.138067,-82.333513,3,2,0,2,...,29,7,2016,0,0,0,170000,35,7.377134,20
4,5,Sold,9203 CELEBRATION CT,CELEBRATION CT,28.129131,-82.340659,4,2,1,3,...,30,9,2016,0,1,0,170000,417,7.741968,23


In [8]:
model1 = smf.ols(formula='pricesold ~ Beds + bathstotal + log_sqft + splsale_BankOwnedREO', data=df).fit()

display(model1.summary())

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              pricesold   R-squared:                       0.757
Model:                            OLS   Adj. R-squared:                  0.755
Method:                 Least Squares   F-statistic:                     368.8
Date:                Wed, 26 Feb 2025   Prob (F-statistic):          7.15e-144
Time:                        20:55:07   Log-Likelihood:                -6043.9
No. Observations:                 478   AIC:                         1.210e+04
Df Residuals:                     473   BIC:                         1.212e+04
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
========================================================================================
                           coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------
Intercept            -2.465e+06   1.59e+05    -15.472      0.000   -2.78e+06   -2.15e+06
Beds                 -1.872e+04   7124.129     -2.628      0.009   -3.27e+04   -4723.479
bathstotal            5.922e+04   7315.961      8.095      0.000    4.48e+04    7.36e+04
log_sqft              3.457e+05   2.35e+04     14.692      0.000    2.99e+05    3.92e+05
splsale_BankOwnedREO -6.232e+04   1.73e+04     -3.611      0.000   -9.62e+04   -2.84e+04
==============================================================================
Omnibus:                      198.646   Durbin-Watson:                   1.124
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             1633.387
Skew:                           1.585   Prob(JB):                         0.00
Kurtosis:                      11.483   Cond. No.                         435.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [9]:
model2 = smf.ols(formula='pricesold ~ lat + long + yrblt + garages + pool_None + ds_moy', data=df).fit()

display(model2.summary())

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              pricesold   R-squared:                       0.535
Model:                            OLS   Adj. R-squared:                  0.530
Method:                 Least Squares   F-statistic:                     90.49
Date:                Wed, 26 Feb 2025   Prob (F-statistic):           3.09e-75
Time:                        20:55:07   Log-Likelihood:                -6199.0
No. Observations:                 478   AIC:                         1.241e+04
Df Residuals:                     471   BIC:                         1.244e+04
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept   -8.54e+08    8.8e+07     -9.700      0.000   -1.03e+09   -6.81e+08
lat         9.323e+06   1.43e+06      6.520      0.000    6.51e+06    1.21e+07
long       -6.905e+06    7.3e+05     -9.462      0.000   -8.34e+06   -5.47e+06
yrblt       1.163e+04   2343.949      4.961      0.000    7021.293    1.62e+04
garages     1.134e+05   1.03e+04     11.002      0.000    9.31e+04    1.34e+05
pool_None  -4.681e+04   1.35e+04     -3.472      0.001   -7.33e+04   -2.03e+04
ds_moy     -1956.6291   1545.612     -1.266      0.206   -4993.778    1080.520
==============================================================================
Omnibus:                      245.086   Durbin-Watson:                   0.944
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             2081.315
Skew:                           2.059   Prob(JB):                         0.00
Kurtosis:                      12.357   Cond. No.                     3.68e+07
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 3.68e+07. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [10]:
model3 = smf.ols(formula='pricesold ~ long + yrblt + splsal_shortsale + pool_None + roof_Tile + house_age', data=df).fit()

display(model3.summary())

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              pricesold   R-squared:                       0.458
Model:                            OLS   Adj. R-squared:                  0.451
Method:                 Least Squares   F-statistic:                     66.38
Date:                Wed, 26 Feb 2025   Prob (F-statistic):           1.26e-59
Time:                        20:55:07   Log-Likelihood:                -6235.8
No. Observations:                 478   AIC:                         1.249e+04
Df Residuals:                     471   BIC:                         1.251e+04
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
====================================================================================
                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------
Intercept         -6.07e+08   5.62e+07    -10.809      0.000   -7.17e+08   -4.97e+08
long             -6.392e+06   6.49e+05     -9.841      0.000   -7.67e+06   -5.12e+06
yrblt             4.046e+04   4512.282      8.967      0.000    3.16e+04    4.93e+04
splsal_shortsale -1.188e+05   4.03e+04     -2.948      0.003   -1.98e+05   -3.96e+04
pool_None        -6.685e+04   1.44e+04     -4.629      0.000   -9.52e+04   -3.85e+04
roof_Tile         9.943e+04   1.09e+04      9.099      0.000     7.8e+04    1.21e+05
house_age         1.605e+04   3777.277      4.248      0.000    8622.636    2.35e+04
==============================================================================
Omnibus:                      287.987   Durbin-Watson:                   0.770
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             3115.078
Skew:                           2.444   Prob(JB):                         0.00
Kurtosis:                      14.512   Cond. No.                     2.17e+07
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 2.17e+07. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [11]:
stargazer = Stargazer([model1, model2, model3])
html = stargazer.render_html()

display(HTML(html))

In [12]:
amodel1 = smf.ols(formula='adom_agentdaysonmarket ~ Beds + sqft + log_sqft + splsale_BankOwnedREO', data=df).fit()

display(amodel1.summary())

<class 'statsmodels.iolib.summary.Summary'>
"""
                              OLS Regression Results                              
==================================================================================
Dep. Variable:     adom_agentdaysonmarket   R-squared:                       0.194
Model:                                OLS   Adj. R-squared:                  0.187
Method:                     Least Squares   F-statistic:                     28.46
Date:                    Wed, 26 Feb 2025   Prob (F-statistic):           3.34e-21
Time:                            20:55:07   Log-Likelihood:                -2723.1
No. Observations:                     478   AIC:                             5456.
Df Residuals:                         473   BIC:                             5477.
Df Model:                               4                                         
Covariance Type:                nonrobust                                         
========================================================================================
                           coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------
Intercept             1674.2573    347.404      4.819      0.000     991.612    2356.903
Beds                    -1.7636      6.814     -0.259      0.796     -15.154      11.627
sqft                     0.1186      0.016      7.295      0.000       0.087       0.151
log_sqft              -244.7614     50.602     -4.837      0.000    -344.194    -145.329
splsale_BankOwnedREO   -20.0561     16.595     -1.209      0.227     -52.665      12.552
==============================================================================
Omnibus:                      199.655   Durbin-Watson:                   1.901
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             1039.939
Skew:                           1.770   Prob(JB):                    1.51e-226
Kurtosis:                       9.299   Cond. No.                     2.93e+05
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 2.93e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [32]:
amodel2 = smf.ols(formula='adom_agentdaysonmarket ~ long + yrblt + lotsqft + bathsfull', data=df).fit()

display(amodel2.summary())

<class 'statsmodels.iolib.summary.Summary'>
"""
                              OLS Regression Results                              
==================================================================================
Dep. Variable:     adom_agentdaysonmarket   R-squared:                       0.099
Model:                                OLS   Adj. R-squared:                  0.091
Method:                     Least Squares   F-statistic:                     12.98
Date:                    Wed, 26 Feb 2025   Prob (F-statistic):           4.87e-10
Time:                            21:06:20   Log-Likelihood:                -2749.7
No. Observations:                     478   AIC:                             5509.
Df Residuals:                         473   BIC:                             5530.
Df Model:                               4                                         
Covariance Type:                nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept   1.592e+04   4.27e+04      0.373      0.709   -6.79e+04    9.98e+04
long         241.3950    493.477      0.489      0.625    -728.283    1211.073
yrblt          1.9847      1.741      1.140      0.255      -1.436       5.406
lotsqft        0.0027      0.001      3.494      0.001       0.001       0.004
bathsfull     11.3531      7.357      1.543      0.123      -3.103      25.809
==============================================================================
Omnibus:                      236.187   Durbin-Watson:                   1.807
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             1469.825
Skew:                           2.092   Prob(JB):                         0.00
Kurtosis:                      10.503   Cond. No.                     1.55e+08
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.55e+08. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [22]:
amodel3 = smf.ols(formula='adom_agentdaysonmarket ~ long + house_age + roof_Tile', data=df).fit()

display(amodel3.summary())

<class 'statsmodels.iolib.summary.Summary'>
"""
                              OLS Regression Results                              
==================================================================================
Dep. Variable:     adom_agentdaysonmarket   R-squared:                       0.069
Model:                                OLS   Adj. R-squared:                  0.063
Method:                     Least Squares   F-statistic:                     11.73
Date:                    Wed, 26 Feb 2025   Prob (F-statistic):           2.01e-07
Time:                            21:02:14   Log-Likelihood:                -2757.5
No. Observations:                     478   AIC:                             5523.
Df Residuals:                         474   BIC:                             5540.
Df Model:                               3                                         
Covariance Type:                nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept  -8.505e+04   3.57e+04     -2.380      0.018   -1.55e+05   -1.48e+04
long       -1035.3313    434.156     -2.385      0.017   -1888.440    -182.223
house_age     -6.6923      1.303     -5.137      0.000      -9.252      -4.132
roof_Tile     15.4724      7.200      2.149      0.032       1.324      29.621
==============================================================================
Omnibus:                      278.129   Durbin-Watson:                   1.760
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             2204.309
Skew:                           2.467   Prob(JB):                         0.00
Kurtosis:                      12.292   Cond. No.                     8.58e+05
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 8.58e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [33]:
astargazer = Stargazer([amodel1, amodel2, amodel3])
ahtml = astargazer.render_html()

display(HTML(ahtml))